In [26]:
import plyvel
import json

articles = []
try:
    db = plyvel.DB("/home/mag/work/emo/medium-sep18")
    for url, bts in db:
        a = json.loads(bts)
        a["url"] = url.decode()
        articles.append(a)
finally:
    db.close()
    
len(articles)

144212

In [93]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
import spacy
import numpy as np
np.random.seed(2018)


import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /home/mag/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [99]:
#nlp = spacy.load('en')
#lemm = Lemmatizer(lookup=lookup)

stemmer = SnowballStemmer("english")

def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

def tokenize(doc):
    result = []
    for token in gensim.utils.simple_preprocess(doc["body"]):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result 

# spacy is waaaay slower here
#    doc = nlp(doc["body"])
#    return [tok.text for tok in doc if tok.is_alpha and not tok.is_stop and len(tok.text) > 2]

In [100]:
tokenize(articles[0])

['kaniko',
 'project',
 'build',
 'googl',
 'allow',
 'build',
 'docker',
 'imag',
 'privileg',
 'dind',
 'docker',
 'docker',
 'help',
 'build',
 'pipelin',
 'base',
 'contain',
 'run',
 'kubernet',
 'probabl',
 'common',
 'solut',
 'build',
 'docker',
 'imag',
 'docker',
 'contain',
 'technic',
 'contain',
 'base',
 'librari',
 'docker',
 'imag',
 'contain',
 'docker',
 'talk',
 'host',
 'docker',
 'daemon',
 'docker',
 'build',
 'process',
 'requir',
 'talk',
 'host',
 'docker',
 'daemon',
 'mount',
 'docker',
 'daemon',
 'contain',
 'contain',
 'privileg',
 'mode',
 'privileg',
 'mode',
 'mount',
 'docker',
 'daemon',
 'lead',
 'secur',
 'issu',
 'allow',
 'environ',
 'kaniko',
 'dind',
 'depend',
 'docker',
 'daemon',
 'allow',
 'build',
 'docker',
 'imag',
 'contain',
 'userspac',
 'extern',
 'depend',
 'privileg',
 'kaniko',
 'document',
 'exampl',
 'kaniko',
 'gitlab',
 'build',
 'pipelin',
 'pipelin',
 'document',
 'kaniko',
 'base',
 'kaniko',
 'project',
 'executor',
 'imag'

In [113]:
proc_docs = [tokenize(a) for a in articles]
proc_docs[:3]

[['kaniko',
  'project',
  'build',
  'googl',
  'allow',
  'build',
  'docker',
  'imag',
  'privileg',
  'dind',
  'docker',
  'docker',
  'help',
  'build',
  'pipelin',
  'base',
  'contain',
  'run',
  'kubernet',
  'probabl',
  'common',
  'solut',
  'build',
  'docker',
  'imag',
  'docker',
  'contain',
  'technic',
  'contain',
  'base',
  'librari',
  'docker',
  'imag',
  'contain',
  'docker',
  'talk',
  'host',
  'docker',
  'daemon',
  'docker',
  'build',
  'process',
  'requir',
  'talk',
  'host',
  'docker',
  'daemon',
  'mount',
  'docker',
  'daemon',
  'contain',
  'contain',
  'privileg',
  'mode',
  'privileg',
  'mode',
  'mount',
  'docker',
  'daemon',
  'lead',
  'secur',
  'issu',
  'allow',
  'environ',
  'kaniko',
  'dind',
  'depend',
  'docker',
  'daemon',
  'allow',
  'build',
  'docker',
  'imag',
  'contain',
  'userspac',
  'extern',
  'depend',
  'privileg',
  'kaniko',
  'document',
  'exampl',
  'kaniko',
  'gitlab',
  'build',
  'pipelin',
  '

In [114]:
dct = gensim.corpora.Dictionary(proc_docs)
count = 0
for k, v in dct.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 abl
1 allow
2 argument
3 authent
4 base
5 binari
6 brief
7 build
8 call
9 common
10 config


In [115]:
dct.filter_extremes(no_below=5, no_above=0.5, keep_n=100000)

In [116]:
bow_corpus = [dct.doc2bow(doc) for doc in proc_docs]

In [117]:
from gensim import corpora, models
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]

In [119]:
lda_model = gensim.models.LdaMulticore(corpus_tfidf, num_topics=10, id2word=dct, passes=5, workers=4)
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

Topic: 0 Word: 0.004*"life" + 0.003*"peopl" + 0.003*"learn" + 0.003*"feel" + 0.003*"thing" + 0.003*"think" + 0.003*"write" + 0.002*"student" + 0.002*"want" + 0.002*"person"
Topic: 1 Word: 0.044*"episod" + 0.033*"season" + 0.010*"watch" + 0.008*"onlin" + 0.007*"https" + 0.006*"seri" + 0.005*"horror" + 0.005*"http" + 0.004*"stream" + 0.004*"video"
Topic: 2 Word: 0.007*"code" + 0.006*"data" + 0.006*"file" + 0.004*"function" + 0.004*"test" + 0.004*"server" + 0.003*"applic" + 0.003*"user" + 0.003*"compon" + 0.003*"python"
Topic: 3 Word: 0.005*"blockchain" + 0.004*"market" + 0.004*"token" + 0.003*"compani" + 0.003*"busi" + 0.003*"data" + 0.003*"technolog" + 0.003*"user" + 0.003*"platform" + 0.003*"custom"
Topic: 4 Word: 0.006*"bybit" + 0.004*"terraform" + 0.003*"mithril" + 0.003*"trustedcar" + 0.002*"thecryptovers" + 0.002*"flex" + 0.002*"blackrock" + 0.002*"pokemon" + 0.002*"excraft" + 0.002*"ganesh"
Topic: 5 Word: 0.003*"women" + 0.002*"polit" + 0.002*"trump" + 0.002*"state" + 0.002*"ameri